# Clase Nº 4

**Plan de la clase:**  
**(1)** Preguntas sobre la clase asincrónica - Revisar el cuestionario. <br>
**(2)** Comentarios sobre la modalidad de entrega del TP4. <br>
**(3)** Repasar `pandas` y `seaborn` (y widgets) <br>
**(4)** Repasar expresiones regulares (y widgets) <br>
~~**(5)** _Fuzzy matching_~~ (lo dejamos para la siguiente clase)

### Preguntas sobre la clase asincrónica / cuestionario

**Ayudamemoria**: <br>
`\d`: match cuando el string contiene digitos (equivalente a [0-9]) <br>
`\D`: match cuando el string NO contiene digitos <br>
`\s`: match cuando el string contiene un espacio en blanco <br>
`\S`: match cuando el string NO contiene un espacio en blanco <br>
`\w`: match cuando el string contiene caracteres de alfanuméricos o "\_" <br>
`\W`: match cuando el string NO contiene caracteres de alfanuméricos o "\_" <br>

Revisemos la pregunta 3 del cuestionario:

3) Escribir una expresión regular que matchee con:

a.  un string que _contiene_ una letra `a` seguida de cualesquiera  caracteres, y termina en una letra `b`.

In [ ]:
import re

In [ ]:
string = "12ab091a00abaaba---b"

In [ ]:
varias_regex = [
  "a.*b", # correcto
  "a.+b", # correcto, si interpretan que debe haber 1 o más "cualesquiera caracteres", esta sería la expresión regular correcta.
  "a.*?b", # el ? permite que re.findall encuentre submatches dentro de un match grande (por defecto devuelve el grande)
  "a.+?b", # ídem arriba, pero no encuentre el primer match, "ab", porque no hay caracteres entre la "a" y la "b
  "a\w*b", # \w sólo tiene en cuenta caracteres alfanuméricos
  "a\w+b", # ídem arriba
  "[a].*[b]", # los corchetes alrededor de un solo caracter no son necesarios  
]

for regex in varias_regex:
    print(f"{regex}: {re.findall(regex, string)}")

b. un string que _contiene_ una o más letras `a` seguidas inmediatamente de cero o más dı́gitos.

In [ ]:
regex = "a+\d*"
# regex1 = "a+[0-9]*" # equivalente
# regex2 = "a+[0123456789]*" # otra regex equivalente

In [ ]:
algunos_strings = ["asaas0000", "a0000", "aba0000"]

for string in algunos_strings:
    print(re.findall(regex, string))

___

## Modalidad de entrega del TP Nº4

El próximo TP va a ser entregado nuevamente a través de GitHub, pero en este caso va ser **grupal**.

Un miembro del grupo debe crear un repositorio en GitHub como se hizo en el TP3, y agregar como colaboradores a sus dos compañeros y a los docentes.

Un requisito va ser que cada miembro del grupo haga al menos un commit, con contribuciones sustanciales.
Para esto es importante que tengan configurado su nombre de usuario de `git` con el mismo nombre que el de GitHub, así los podemos identificar fácilmente.

___

## Pandas y Seaborn

### Reformateo de `DataFrame`'s

Sigamos trabajando con el DataFrame de centrales eléctricas.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("potencia_instalada_mod.csv", sep="|", encoding="latin1")

In [ ]:
df.region.value_counts()

In [ ]:
df.fuente_generacion.value_counts()

In [ ]:
df.head(2)

1. Primero, calculemos la potencia total instalada en cada región en un período dado, haciendo un `groupby` y usando la función de agregación adecuada.

In [ ]:
df_region = df.groupby(["region", "periodo"]).agg({"potencia_instalada_mw": "sum"}).reset_index()

# Pregunta de Franco: 
# para generarnos un DataFrame de una sola columna (en lugar de una Series),
# pasamos una lista con una sola columna:
df_region["potencia_instalada_mw"] # esto devuelve una Series
df_region[["potencia_instalada_mw"]] # esto devuelve un DataFrame

2. Luego, formateemos el `DataFrame` de manera que los nombres de las regiones sean ahora los nombres de nuevas columnas, y el contenido de la celda sea la potencia instalada en esa región en el período correspondiente (dado en filas). <br>
Hay que ir de un formato "largo" a uno "ancho".
Pueden consultar la documentación acerca de reformateo de dataframes aquí: https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html, para ver qué método necesitarían usar.

In [ ]:
df_region_pivot = df_region.pivot(index="periodo", columns="region", values="potencia_instalada_mw")
df_region_pivot.drop("(en blanco)", axis=1, inplace=True) # eliminamos una columna indeseada

# Durante la clase me olvidé de agregar el string de formateo!
# Eso hizo que las fechas no se parseen correctamente
df_region_pivot.index = pd.to_datetime(df_region_pivot.index, format="%d/%m/%Y %H:%M")
df_region_pivot.sort_index()

3. Escribamos una función que reciba como argumento el `DataFrame` formateado como en el punto anterior y un string que represente una región, y utilicemos `seaborn` para hacer un gráfico de la potencia instalada para esa región. Pueden usar, por ejemplo, el tipo de plot `lineplot` (el argumento `data` de la función debe ser el `DataFrame` formateado). Pueden usar los comandos vistos en matplotlib para formatear un plot de Seaborn.

In [ ]:
import seaborn as sns

In [ ]:
def plot(df, region):
    sns.lineplot(data=df, x="periodo", y=region)

In [ ]:
# Examinemos los valores para una región
df_region_pivot[["BUENOS AIRES"]].sort_index().head()

Podemos obtener una lista de las regiones, simplemente obteniendo las columnas del dataframe pivoteado

In [ ]:
regiones = df_region_pivot.columns.to_list()
regiones

In [ ]:
plot(df_region_pivot, "CENTRO")

4. Finalmente, usemos un widget reactivo para elegir qué región graficar. Pueden consultar la documentación de widget acá: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

region_w = widgets.Select(options=df_region_pivot.columns)

Escribimos una función `f` que es una aplicación parcial de la función de arriba, en la cual fijamos uno de los argumentos (el dataframe), de manera que el argumento sea sólo la región. 

Lo pueden pensar similar a lo que ocurre cuando tienen una función de dos variables $f(x,y)$, y definen una nueva al fijar uno de sus argumentos, por ejemplo $g(y)=f(2,y)$ ó $h(x)=f(x,0)$.

In [ ]:
# Pueden usar
sns.set(rc={'figure.figsize':(10, 4)})
# para cambiar el tamaño de la figura

def f(region):
    plot(df_region_pivot, region)
    
interact(f, region=region_w);

La siguiente es otra forma de hacerlo, que usa "decoradores" (esto no lo vimos en clase, pero ahorra un poco de código):

In [ ]:
@interact
def f(region=region_w):
    plot(df_region_pivot, region)

___________________________________________________

## Expresiones regulares

In [ ]:
import re

#### Ejercicio:
Vamos a seguir trabajando con el Martín Fierro. <br>
Primero repitamos el proceso de _parseo_ realizado durante la clase asincrónica

In [ ]:
with open("martin_fierro.txt") as archivo:
    martin_fierro = archivo.read()

In [ ]:
def parsear_martin_fierro(martin_fierro):
    partes = re.split("[IVX]+ - .*\n", martin_fierro)
    partes.pop(0)
    
    estrofas = partes.copy()
    for i in range(len(estrofas)):
        estrofas[i] = re.split("\n\d+\n", estrofas[i])
        estrofas[i].pop(0)
    
    versos = estrofas.copy()
    for i in range(len(versos)):
        for j in range(len(versos[i])):
            versos[i][j] = versos[i][j].split("\n")
            del versos[i][j][-1] # Agregué esto para eliminar el string vacío al final de cada estrofa
    
    # Quitamos
    versos[-1][-1] = versos[-1][-1][0:6]
    
    return versos

In [ ]:
todos_los_versos = parsear_martin_fierro(martin_fierro)
todos_los_versos

**Consigna**

Construir un _widget_ reactivo tal que uno pueda escribir una expresión regular y, si tal expresión regular matchea con alguna parte del libro, muestre todos los números de parte, estrofa y verso en los que aparece. Si no aparecen en ningún lado, debe imprimir un mensaje diciendo `"La expresión regular <EXPRESIÓN REGULAR> no matchea con ningún verso del Martín Fierro.`

_Ayudas_: 
- El widget que deberían usar es de tipo `TextArea`. 
- Dentro de la función que se le pasa a `interact` debería haber loops `for` anidados (como se hizo en la clase asincrónica), para iterar sobre las estructuras jerárquicas que componen el libro.
- Recordar, por ejemplo, el método `enumerate` que devuelve tanto los índices como los elementos sobre los cuales se itera (no es estrictamente necesario usarlo, pero puede ser útil).


In [ ]:
regex_w = widgets.Textarea()

In [ ]:
def encontrar_ocurrencia(patron):
    
    regex = re.compile(patron)
    
    no_match = True
    
    for i, parte in enumerate(todos_los_versos):    
        for j, estrofa in enumerate(parte):
            for k, verso in enumerate(estrofa):
                if regex.match(verso):
                    no_match = False
                    print(f"parte {i}, estrofa {j}, verso {k}: {verso}")
                    
    if no_match:
        print(f"La expresión regular {patron} no matchea con ningún verso del Martín Fierro.")

In [ ]:
interact(encontrar_ocurrencia, patron=regex_w);

_______________________________________________________________

## _Fuzzy matching_ (esto lo veremos la próxima clase)

_Fuzzy matching_ refiere al concepto de encontrar correspondencias entre strings que no son exactamente iguales.

Instalemos el módulo `fuzzywuzzy` con `pip install fuzzywuzzy`.

In [ ]:
from fuzzywuzzy import fuzz, process

### Distancia de Levenshtein

Es la cantidad mínima de caracteres que deben editarse en un string para equipararlo a otro. Una edición puede ser una inserción, una eliminación o una substitución.

`fuzzywuzzy` es una librería de Python que tiene funciones para medir distancias entre `string`'s. Estima la distancia y devuelve un indicador de similitud entre 0 (menor similitud) y 100 (máxima similitud).

Documentación (es algo básica): https://github.com/seatgeek/fuzzywuzzy

In [ ]:
# Simple Ratio
fuzz.ratio("Energía Termica", "Energía Térmica")

In [ ]:
# Partial Ratio
fuzz.partial_ratio("Energía Termica", "La Energía Térmica es la más común")

In [ ]:
# Token set ratio
fuzz.token_sort_ratio("La Energía Térmica es la más común", "La más común es Energía Térmica la")

In [ ]:
# Token set ratio
fuzz.token_set_ratio("La Energía Térmica es la más común", "La más más común es Energía Térmica la la ")

In [ ]:
from fuzzywuzzy import process

In [ ]:
?process.extractOne

In [ ]:
fuentes_validas = ['Térmica','Renovable','Hidráulica','Nuclear']

In [ ]:
process.extractOne("Temica", fuentes_validas)

In [ ]:
process.extractOne("Temica", fuentes_validas)[0]

**Aplicación**: examinemos la tabla de potencia instalada:

In [ ]:
df = pd.read_csv("Material para Zoom/potencia_instalada_con_errores.csv", sep="|", encoding="latin1")

In [ ]:
df['fuente_generacion'].value_counts()

In [ ]:
def limpiar_fuente(fuente_generacion, fuentes_validas=['Térmica','Renovable','Hidráulica','Nuclear']):
    '''
    Función toma como input un string que identifica una fuente
    de generación eléctrica mal escrita y la reemplaza por la fuente
    válida más parecida.
    '''
            
    fuente_procesada = process.extractOne(fuente_generacion, fuentes_validas)[0]    
    return fuente_procesada

In [ ]:
df['fuente_generacion'] = df['fuente_generacion'].apply(limpiar_fuente)

In [ ]:
df['fuente_generacion'].value_counts()